In [ ]:
from pathlib import Path
import numpy as np
from rosbag import Bag
from pathlib import Path
from scipy.interpolate import interp1d

def get_bag(directory, pattern):
    return [str(s) for s in Path(directory).rglob(pattern)][0]

ptp_topics = {
    "/gt_box/health_status/jetson/status" : ("*_jetson_utils.bag", ["offset_ptp4l_mgbe0", "offset_phc2sys_mgbe0"]),
    "/gt_box/health_status/nuc/status": ("*nuc_utils.bag", ["offset_ptp4l_enp45s0", "offset_phc2sys_enp45s0",  "offset_phc2sys_enp46s0"]),
}

# Example usage
directory = "/mission_data"  # Path where ROS bags are stored


offsets = {}
times = {}

for topic, info in ptp_topics.items():
    pattern = info[0]
    bag_file = get_bag(directory, pattern)
    fields = info[1]
    
    for field in fields:
        offsets[field] = []
        times[field] = []
    
    with Bag(bag_file, 'r') as bag:
        for _topic_name, msg, t in bag.read_messages(topics=[topic]):
            for field in fields:    
                offsets[field].append(getattr(msg,field))
                times[field].append(t.to_sec())
            

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Start plotting
plt.figure(figsize=(10, 6))

k = 0
# Loop through each key and array in the dictionary
for ptp_name, ptp_offsets in offsets.items():
    ptp_offset = np.array(ptp_offsets)
    valid = ptp_offset != "error_reading_offset"
    res = ptp_offset[valid]
    tim = np.array(times[ptp_name])[valid]
    # window_size = 100
    # res = np.convolve(np.float32( res) , np.ones((window_size,)) / window_size, mode='valid')
    plt.plot( res  , label=ptp_name)
    k += 1
    
    # Add labels and title
    plt.xlabel('Message Number - 1 Message per second')
    plt.ylabel('PTP offsets in [ns]')
    plt.title('PTP Sync over Time')
    plt.legend()
    plt.show()